In [33]:
import aeon
from aeon.classification.distance_based import KNeighborsTimeSeriesClassifier
from sklearn.preprocessing import StandardScaler
import json
import matplotlib.pyplot as plt
import numpy as np

In [10]:
# load original dataset
train = np.loadtxt("../../../../Data/NASA_KeplerV2/datasets/classification/KeplerBinaryOriBal_TRAIN.txt")
test = np.loadtxt("../../../../Data/NASA_KeplerV2/datasets/classification/KeplerBinaryOriBal_TEST.txt")

In [50]:
# load fold indexes
with open("../../../../FinalBenchmarks/NASA_KeplerV2/classification/Python/KeplerBenchFoldsPython.json", "rb") as f:
    folds = json.load(f)

In [51]:
folds['0']['train']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [27]:
X_train_original = train[:, 1:101]
y_train_original = train[:, 0].astype(int)
X_test_original = test[:, 1:101]
y_test_original = test[:, 0].astype(int)

In [56]:
# merge
Xs = np.vstack([X_train_original, X_test_original])
Xs.shape

(402, 100)

In [59]:
ys = np.hstack([y_train_original, y_test_original])
ys.shape

(402,)

In [73]:
# main loop 
per_fold_accs = []
for fold in range(0, 30):
    train_idxs = folds[f'{fold}']['train']
    test_idxs = folds[f'{fold}']['test']
    X_train_fold = Xs[train_idxs]
    y_train_fold = ys[train_idxs]
    X_test_fold = Xs[test_idxs]
    y_test_fold = ys[test_idxs]
    # scale the data
    zs = StandardScaler().fit(X_train_fold)
    X_train_fold_scaled = zs.transform(X_train_fold)
    X_test_fold_scaled = zs.transform(X_test_fold)
    # fit the knn classifier
    nndtw = KNeighborsTimeSeriesClassifier(distance="dtw", n_neighbors=1).fit(X_train_fold_scaled, y_train_fold)
    score = nndtw.score(X_test_fold_scaled, y_test_fold)
    print(f"Fold {fold}, score: {score}")
    per_fold_accs.append(score)

    

Fold 0, score: 0.9012345679012346
Fold 1, score: 0.9146341463414634
Fold 2, score: 0.8902439024390244
Fold 3, score: 0.9024390243902439
Fold 4, score: 0.8902439024390244
Fold 5, score: 0.9024390243902439
Fold 6, score: 0.926829268292683
Fold 7, score: 0.8536585365853658
Fold 8, score: 0.9024390243902439
Fold 9, score: 0.8902439024390244
Fold 10, score: 0.8658536585365854
Fold 11, score: 0.8902439024390244
Fold 12, score: 0.8902439024390244
Fold 13, score: 0.8658536585365854
Fold 14, score: 0.7926829268292683
Fold 15, score: 0.8902439024390244
Fold 16, score: 0.9146341463414634
Fold 17, score: 0.8536585365853658
Fold 18, score: 0.9024390243902439
Fold 19, score: 0.9390243902439024
Fold 20, score: 0.8658536585365854
Fold 21, score: 0.8658536585365854
Fold 22, score: 0.9024390243902439
Fold 23, score: 0.9390243902439024
Fold 24, score: 0.926829268292683
Fold 25, score: 0.8780487804878049
Fold 26, score: 0.8902439024390244
Fold 27, score: 0.8902439024390244
Fold 28, score: 0.93902439024390

In [74]:
np.mean(per_fold_accs)

0.8918297701495533

In [76]:
np.savetxt("nn_dtw_kepler_30fold_results.txt", per_fold_accs)